In [1]:
using System;
using System.Collections.Concurrent;
using System.Threading;

public interface ICommand
{
    void Execute();
}

public class ServerThread
{
    private BlockingCollection<ICommand> commandQueue = new BlockingCollection<ICommand>();
    private Thread thread;
    private bool isRunning = true;
    private bool isSoftStopping = false;
    private bool isHardStopping = false;
    private ConcurrentQueue<ICommand> schedulerQueue = new ConcurrentQueue<ICommand>();

    public ServerThread()
    {
        thread = new Thread(Run);
        thread.Start();
    }

    public void AddCommand(ICommand command)
    {
        commandQueue.Add(command);
    }

    private void Run()
    {
        while (isRunning)
        {
            ICommand command;
            if (commandQueue.TryTake(out command, Timeout.Infinite))
            {
                if (!isSoftStopping && !isHardStopping)
                {
                    try
                    {
                        command.Execute();
                    }
                    catch (Exception ex)
                    {
                        // Обработка исключения
                        Console.WriteLine($"Исключение в команде: {ex.Message}");
                    }
                }
                else
                {
                    commandQueue.Add(command);
                }
            }
            else if (isSoftStopping)
            {
                isRunning = false;
            }
            else
            {
                Thread.Sleep(1);
            }

            if (schedulerQueue.TryDequeue(out command))
            {
                commandQueue.Add(command);
            }
        }
    }

   public void SoftStop()
    {
        
        isSoftStopping = true;
    }

      public void HardStop()
    {
       

        isRunning = false;
        thread.Abort();
    }
}

public class LongCommand : ICommand
{
    private readonly int iterations;
    private int currentIteration;
    private readonly ServerThread serverThread;

    public LongCommand(int iterations, ServerThread serverThread)
    {
        this.iterations = iterations;
        this.serverThread = serverThread;
    }

    public void Execute()
    {
        for (int i = 0; i < 10; i++)
        {
            Console.WriteLine($"Команда {GetHashCode()} - итерация {currentIteration + i + 1} из {iterations}");
            Thread.Sleep(100);
        }

        currentIteration += 10;

        if (currentIteration < iterations)
        {
            serverThread.AddCommand(this); 
        }
    }
}

In [2]:
var serverThread = new ServerThread();

serverThread.AddCommand(new LongCommand(150, serverThread));
serverThread.AddCommand(new LongCommand(20, serverThread));
serverThread.AddCommand(new LongCommand(400, serverThread));

Thread.Sleep(5000);

serverThread.SoftStop();

Console.WriteLine("Основной поток завершается.");

Команда 42057683 - итерация 1 из 150
Команда 42057683 - итерация 2 из 150
Команда 42057683 - итерация 3 из 150
Команда 42057683 - итерация 4 из 150
Команда 42057683 - итерация 5 из 150
Команда 42057683 - итерация 6 из 150
Команда 42057683 - итерация 7 из 150
Команда 42057683 - итерация 8 из 150
Команда 42057683 - итерация 9 из 150
Команда 42057683 - итерация 10 из 150
Команда 39770063 - итерация 1 из 20
Команда 39770063 - итерация 2 из 20
Команда 39770063 - итерация 3 из 20
Команда 39770063 - итерация 4 из 20
Команда 39770063 - итерация 5 из 20
Команда 39770063 - итерация 6 из 20
Команда 39770063 - итерация 7 из 20
Команда 39770063 - итерация 8 из 20
Команда 39770063 - итерация 9 из 20
Команда 39770063 - итерация 10 из 20
Команда 17533491 - итерация 1 из 400
Команда 17533491 - итерация 2 из 400
Команда 17533491 - итерация 3 из 400
Команда 17533491 - итерация 4 из 400
Команда 17533491 - итерация 5 из 400
Команда 17533491 - итерация 6 из 400
Команда 17533491 - итерация 7 из 400
Команда 1